In [84]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.metrics.pairwise import cosine_similarity

In [85]:
data = pd.read_csv('./conversations.csv')
data

,message,response
0,Hello,"Hi, how are you?"
1,"Hey, how is it going?","Great, how about you?"
2,How are you doing?,"Very well, what about you?"
3,What’s up?,"Nothing much, what about you?"
4,"Hi, nice to meet you","Thank you, nice to meet you too"
5,Who are you?,Who? Who is but a form following the function ...
6,Can I ask you a question?,"Sure, ask away"
7,What are you?,I am a chatbot designed by a graduate student ...
8,What are your interests,I am interested in all kinds of things. We can...
9,Where are you?,I am on the internet


In [86]:
class TFIDFPredictor:
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix_train = None

    def train(self, data):
        self.vectorizer.fit(data.message.values)
        self.tfidf_matrix_train = self.vectorizer.transform(data.message.values) 
        
    def predict(self, context):
        # Convert context and utterances into tfidf vector
        vector_context = self.vectorizer.transform([context])
        #vector_doc = self.vectorizer.transform(utterances)
        vector_doc = self.tfidf_matrix_train
        
        # The dot product measures the similarity of the resulting vectors
        #result = np.dot(vector_doc, vector_context.T).todense()
        #result = np.asarray(result).flatten()
        result = cosine_similarity(vector_doc, vector_context)
        result = np.asarray(result).flatten()
        
        # Sort by top results and return the indices in descending order
        return np.argsort(result, axis=0)[::-1][:5], np.sort(result, axis=0)[::-1][:5]

In [87]:
tfidf = TFIDFPredictor()
tfidf.train(data)

/home/olivier/anaconda3/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [88]:
y_pred, y_values = tfidf.predict('Who made you?')
for idx, (pred, val) in enumerate(zip(y_pred, y_values)):
    print("(%d) %s => %s [%.4f]" % (idx, data.iloc[pred].message, data.iloc[pred].response , val))

(0) Who made you? => A graduate student from the University of Waterloo [1.0000]
(1) Who are you? => Who? Who is but a form following the function of what [0.5679]
(2) Who is your father? => A human [0.2539]
(3) What are you? => I am a chatbot designed by a graduate student from the University of Waterloo [0.1415]
(4) Where are you? => I am on the internet [0.1326]


/home/olivier/anaconda3/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [ ]:
y = [pred.predict(test_df.message[x], test_df.iloc[x,1:].values) for x in range(len(test_df))]